# Carga y limpieza de datos
https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

# Paso 1: Define tus rutas
Primero, definamos las variables para tus carpetas. Esto hace que tu código sea mucho más limpio y fácil de mantener.

In [3]:
# --- 1. Definir Rutas (CORREGIDO) ---

# Ruta base de tu proyecto
BASE_DIR = "/content/drive/My Drive/AI Projects/Customer Churn/"

# Carpeta donde están los datos
DATA_DIR = BASE_DIR + "datasets/"

# Ruta AL ARCHIVO .zip (¡Esta es la corrección!)
# El archivo .zip está DENTRO de la carpeta datasets
ZIP_FILE = DATA_DIR + "brazilian-ecommerce.zip"


: 

# Paso 2: Descomprime el archivo .zip
Ahora, usa un comando de Linux (!unzip) para descomprimir los archivos en la carpeta datasets que acabas de definir.

! le dice a Colab que ejecute un comando de terminal.

-q significa "quiet" (silencioso), para que no te muestre los 100.000 archivos mientras descomprime.

-d especifica el directorio de destino.

In [ ]:
# --- 2. Descomprimir los datos ---

print(f"Descomprimiendo {ZIP_FILE} en {DATA_DIR}...")

# Usamos -n para evitar descomprimir si ya existen (no sobreescribir)
!unzip -q -n "{ZIP_FILE}" -d "{DATA_DIR}"

print("¡Descompresión completada!")

: 

# Paso 3: Verifica los archivos
Comprobemos que los archivos .csv están donde esperamos

In [ ]:
# --- 3. Verificar los archivos ---
# Deberías ver una lista de todos los .csv (ej: olist_customers_dataset.csv, etc.)

!ls "{DATA_DIR}"

# Paso 4: Cargar y Entender los Datos
1. Cargar todos los CSVs en DataFrames
Vamos a cargar todos los archivos en pandas y a darles nombres lógicos

In [ ]:
# --- 4. Cargar todos los datasets ---
import pandas as pd
import os # Para unir rutas de forma limpia

# Carga todos los archivos .csv en un diccionario de DataFrames
# Esto es más limpio que tener 9 variables separadas

dataframes = {} # Un diccionario para guardar cada df
files_to_load = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_order_payments_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_orders_dataset.csv",
    "olist_products_dataset.csv",
    "olist_sellers_dataset.csv",
    "product_category_name_translation.csv"
]

print("Cargando archivos...")
for file in files_to_load:
    # Quita el ".csv" y el "olist_" para crear un nombre de clave limpio
    key_name = file.replace("olist_", "").replace("_dataset.csv", "").replace(".csv", "")

    file_path = os.path.join(DATA_DIR, file) # os.path.join es más robusto que usar "+"

    try:
        dataframes[key_name] = pd.read_csv(file_path)
        print(f"  ✅ Cargado: {file} (como dataframes['{key_name}'])")
    except FileNotFoundError:
        print(f"  ❌ Error: No se encontró {file_path}. Verifica tus rutas.")

print("\n¡Carga completada!")

# El Primer "Merge" (Unión)
## 1. DataFrame de Pedidos (orders):

In [ ]:
df_orders = dataframes['orders']
print("Columnas de Pedidos:")
print(df_orders.columns.tolist())

## DataFrame de Clientes (customers):

In [ ]:
df_customers = dataframes['customers']
print("\nColumnas de Clientes:")
print(df_customers.columns.tolist())

El customer_id es único por pedido. El customer_unique_id es único por persona. Esta es la clave para identificar a un cliente que vuelve.

Vamos a unirlos:
Un DataFrame que combina la información del pedido (cuándo se hizo, su estado) con la información del cliente (de qué ciudad es, cuál es su ID único).

In [ ]:
# Unir pedidos con clientes usando la clave 'customer_id'
# Usamos un 'left' merge para asegurar que mantenemos todos los pedidos,
# incluso si (hipotéticamente) a alguno le faltara un cliente.

df_master = pd.merge(
    left=df_orders,
    right=df_customers,
    on="customer_id",
    how="left"
)

# Muestra el resultado
print("\n--- DataFrame Maestro (Pedidos + Clientes) ---")
display(df_master.head())

print(f"Filas en Pedidos: {len(df_orders)}")
print(f"Filas en Master:  {len(df_master)}")

# Paso 5: Agregar el Valor del Pedido (Payments)
Aquí nos encontramos con nuestro primer gran desafío de "Big Data" que requiere una decisión estratégica.

El Problema: No podemos hacer un merge simple de df_master con dataframes['order_payments']. ¿Por qué? Porque un solo pedido (order_id) puede tener múltiples métodos de pago. Por ejemplo, un cliente puede pagar 100€ usando 80€ de tarjeta de crédito y 20€ de un vale (voucher). Esto resultaría en dos filas para el mismo pedido en la tabla order_payments.

La Trampa: Si hiciéramos un merge directo, duplicaríamos filas en nuestro df_master, y al sumar el dinero, ¡contaríamos el pedido varias veces!

La Solución Profesional: Primero pre-agregamos la tabla order_payments. La "colapsamos" para que tenga solo una fila por order_id que contenga la suma de todo el valor pagado.

In [ ]:
# --- 5.1: Pre-agregar la tabla de Pagos ---

# Cargar la tabla de pagos desde nuestro diccionario
df_payments = dataframes['order_payments']

print("--- Pagos (Original) ---")
print(f"Filas en 'payments': {len(df_payments)}")
display(df_payments.head()) # Puedes descomentar esto para verla

# Agrupar por 'order_id' y sumar el 'payment_value'
# .reset_index() convierte la salida de groupby de nuevo en un DataFrame
df_payments_agg = df_payments.groupby('order_id')['payment_value'].sum().reset_index()

# Renombramos la columna para que sea más clara
df_payments_agg = df_payments_agg.rename(columns={'payment_value': 'total_payment_value'})

print("\n--- Pagos (Agregados por Pedido) ---")
print(f"Filas en 'payments_agg': {len(df_payments_agg)}")
display(df_payments_agg.head())

# 5.2: Unir los Pagos a df_master
Ahora que tenemos una tabla limpia con una fila por pedido, podemos unirla a df_master de forma segura.

In [ ]:
# --- 5.2: Unir los pagos agregados a nuestro df_master ---

print(f"Filas en df_master (antes del merge): {len(df_master)}")

# Usamos 'how="left"' para mantener todos nuestros pedidos,
# incluso si alguno (hipotéticamente) no tuviera pago registrado.
df_master = pd.merge(
    left=df_master,
    right=df_payments_agg,
    on="order_id",
    how="left"
)

print(f"Filas en df_master (después del merge): {len(df_master)}")
print("\n--- DataFrame Maestro (con info de Pagos) ---")
display(df_master.head())

# Paso 6: Agregar los Productos del Pedido (Items)
Este paso es muy similar al de los pagos y es fundamental que entiendas la estrategia.

El Problema: La tabla dataframes['order_items'] tiene una fila por cada producto dentro de un pedido. Si un cliente compró 3 productos en un solo pedido (order_id), habrá 3 filas en esta tabla para ese pedido.

La Trampa: Si hiciéramos un merge simple de df_master con order_items, nuestro df_master (que con tanto esfuerzo mantuvimos en 99,441 filas) se multiplicaría y tendríamos filas duplicadas de pedidos.

La Solución Profesional: De nuevo, pre-agregamos la tabla order_items. La "colapsaremos" para que tenga solo una fila por order_id que resuma la información de los productos.

In [ ]:
# --- 6.1: Pre-agregar la tabla de Items ---

# Cargar la tabla de items desde nuestro diccionario
df_items = dataframes['order_items']

print("--- Items (Original) ---")
print(f"Filas en 'items': {len(df_items)}")
# La columna 'order_item_id' nos dice el número de item (1, 2, 3...)
# 'price' es el valor de ESE item
# 'freight_value' es el valor del envío de ESE item
display(df_items.head())

# Agrupar por 'order_id' y realizar múltiples agregaciones
df_items_agg = df_items.groupby('order_id').agg(
    # Para el num_items, contamos el 'order_item_id' (es como 1, 2, 3...)
    num_items=('order_item_id', 'count'),

    # Sumamos el precio de todos los items en el pedido
    total_product_value=('price', 'sum'),

    # Sumamos el envío de todos los items en el pedido
    total_freight_value=('freight_value', 'sum')
).reset_index() # .reset_index() convierte la salida de groupby en un DataFrame

print("\n--- Items (Agregados por Pedido) ---")
print(f"Filas en 'items_agg': {len(df_items_agg)}")
display(df_items_agg.head())

## 6.2: Unir los Items a df_master
Ahora que tenemos esta tabla df_items_agg limpia (una fila por pedido), podemos unirla a df_master de forma segura.

In [ ]:
# --- 6.2: Unir los items agregados a nuestro df_master ---

print(f"Filas en df_master (antes del merge): {len(df_master)}")

# Usamos 'how="left"'
df_master = pd.merge(
    left=df_master,
    right=df_items_agg,
    on="order_id",
    how="left"
)

print(f"Filas en df_master (después del merge): {len(df_master)}")
print("\n--- DataFrame Maestro (con info de Items) ---")
display(df_master.head())

# Paso 7: Agregar la Categoría del Producto
El Problema: Un solo pedido (order_id) puede contener múltiples productos (ej. un par de "zapatos" y una "camiseta"). Si intentamos unir los productos, obtendremos múltiples filas para un mismo pedido.

La Decisión Estratégica: Para nuestro df_master (que queremos mantener como una fila por pedido), no podemos listar todas las categorías. Por lo tanto, tomaremos una decisión de negocio: vamos a identificar el producto "principal" de cada pedido y usaremos su categoría.

Nuestra Definición de "Principal": El producto más caro (price) de ese pedido.

## 7.1: Identificar el Producto Principal de cada Pedido
Vamos a crear una tabla temporal que solo contenga el product_id del item más caro de cada order_id.

In [ ]:
# --- 7.1: Identificar el Producto Principal de cada Pedido ---

# Cargar la tabla de items
df_items = dataframes['order_items']

# 1. Ordenar los items por 'order_id' y luego por 'price' (de mayor a menor)
df_items_sorted = df_items.sort_values(by=['order_id', 'price'], ascending=[True, False])

# 2. Quedarse solo con la primera fila de cada 'order_id'.
# Esto nos da el item más caro de cada pedido.
df_main_item = df_items_sorted.drop_duplicates(subset='order_id', keep='first')

# 3. Solo necesitamos estas dos columnas para el 'merge'
df_main_item_lookup = df_main_item[['order_id', 'product_id']]

print("--- Tabla de Búsqueda (Pedido -> Producto Principal) ---")
display(df_main_item_lookup.head())
print(f"Filas en 'main_item_lookup': {len(df_main_item_lookup)}")

## 7.2: Crear una Tabla de Búsqueda de Categorías
Ahora, vamos a crear una "tabla de consulta" que una product_id -> category_name -> category_name_english.

In [ ]:
# --- 7.2: Crear Tabla de Búsqueda () ---

# Cargar las tablas de productos y de traducción
df_products = dataframes['products']
df_translation = dataframes['product_category_name_translation']

# 1. Unir productos con su traducción al inglés
# Solo seleccionamos las columnas que necesitamos para mantenerlo limpio
df_categories_en = pd.merge(
    left=df_products[['product_id', 'product_category_name']],
    right=df_translation,
    on='product_category_name',
    how='left'
)

# 2. Solo necesitamos 'product_id' y el nombre en inglés
df_category_lookup = df_categories_en[['product_id', 'product_category_name_english']]

print("--- Tabla de Búsqueda (Producto -> Categoría EN) ---")
display(df_category_lookup.head())

## 7.3: Unir Todo en el df_master
Ahora tenemos todas las piezas del rompecabezas:

df_master: Nuestro DataFrame principal (una fila por pedido).

df_main_item_lookup: Conecta order_id -> product_id (del item más caro).

df_category_lookup: Conecta product_id -> product_category_name_english.

In [ ]:
# --- 7.3: Unir Categorías al DataFrame Maestro ---

print(f"Filas en df_master (antes): {len(df_master)}")

# 1. Unir df_master con el producto principal de cada pedido
df_master = pd.merge(
    left=df_master,
    right=df_main_item_lookup,
    on='order_id',
    how='left'
)

# 2. Unir el resultado con la tabla de categorías en inglés
df_master = pd.merge(
    left=df_master,
    right=df_category_lookup,
    on='product_id',
    how='left'
)

print(f"Filas en df_master (después): {len(df_master)}")
print("\n--- DataFrame Maestro (Final - con Categorías) ---")
display(df_master.head())

Primero, necesitamos cargar las librerías que usaremos y el df_master que creamos en el notebook anterior.

Importante: Para guardar tu df_master al final del Notebook 01, te recomiendo usar to_parquet() en lugar de .to_csv(). Es mucho más rápido y mantiene los tipos de datos (como fechas) correctamente.

In [17]:
# Guardar en formato Parquet (eficiente)
df_master.to_parquet(DATA_DIR + "df_master_transaccional.parquet")